# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `geemap` module allows interactive analysis and visualization of GEE datasets in a Jupyter environment.
* The `datetime` module supplies classes for manipulating dates and times.
* The `tabulate` module allows the user to display data in a table format.

In [1]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.6 MB/s eta 0:00:00


In [18]:
import ee
import geemap
import datetime
import tabulate

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [3]:
ee.Authenticate()
ee.Initialize(project="...")

# **3) Functions**

Data Processing

In [19]:
def export_tasks_viewer(exportTasksIds, tableFormat: str = "plain"):
  """
  Description:
    Displays a table view which contains useful information about the provided export tasks.

  Notes:
    * Task_Id: The task identifier.
    * Task_State: One of READY, RUNNING, COMPLETED, FAILED, CANCELLED, UNSUBMITTED or UNKNOWN.
    * Task_Type: One of EXPORT_IMAGE, EXPORT_TILES, EXPORT_FEATURES, EXPORT_VIDEO.
    * Task_Attempt: Number of attempts.
    * Task_Description: A human-readable description of the task.
    * Queue_Time: The time that is taken while being in a queue.
    * Execution_Time: The time spent by the servers executing the task.
    * Completion_Time: SUm of queue and execution times.
    * Error_Message: Failure reason. Appears only if state is FAILED. May also include other fields.

  Arguments:
    exportTasksIdsList (list) (mandatory) A list of export task identifiers.
    tableFormat (str) (optional) The table format to use. Defaults to "plain".

  Returns:
    None, displays the export tasks table.
  """
  taskInfo = []
  tableHeaders = [
    "Task_Id", "Task_State", "Task_Type", "Task_Attempt", "Task_Description",
    "Queue_Time", "Execution_Time", "Completion_Time", "Error_Message"
  ]
  tableFormats = tabulate._table_formats.keys()

  if tableFormat not in tableFormats:
    raise ValueError(f"Invalid table format. Choose from: `{tableFormats}`.")

  # Populate taskInfo.
  for exportTaskId in exportTasksIds:

    taskState = ee.data.getTaskStatus(exportTaskId)[0]["state"]
    taskType = ee.data.getTaskStatus(exportTaskId)[0]["task_type"]
    taskDescription = ee.data.getTaskStatus(exportTaskId)[0]["description"]
    startTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["start_timestamp_ms"]/1000.0)
    updateTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["update_timestamp_ms"]/1000.0)
    creationTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["creation_timestamp_ms"]/1000.0)

    queueTime = None
    taskAttempt = None
    executionTime = None
    completionTime = None

    if taskState not in ["READY", "RUNNING"]:
      queueTime = (startTimestamp - creationTimestamp).total_seconds()
      executionTime = (updateTimestamp - startTimestamp).total_seconds()

    if taskState == "COMPLETED":
      taskAttempt = ee.data.getTaskStatus(exportTaskId)[0]["attempt"]
      completionTime = (updateTimestamp - creationTimestamp).total_seconds()

    try:
      errorMessage = ee.data.getTaskStatus(exportTaskId)[0]["error_message"]
    except KeyError:
      errorMessage = None  # This just means that the export task has not failed.

    taskInfo.append([exportTaskId, taskState, taskType, taskAttempt, taskDescription, queueTime, executionTime, completionTime, errorMessage])

  # Table display.
  table = tabulate.tabulate(taskInfo, headers=tableHeaders, tablefmt=tableFormat)
  print(table)

# **4) Parameters**

In [13]:
# Projection of interest.
projectionCRS = "EPSG:4326"
projectionScale = 10

# `Digital Elevation`
demProvider = "USGS"
steepnessThreshold = 8

# `Dynamic World Land Cover`
dwStartDate = "2023-01-01"
dwEndDate = "2023-09-01"

# `Classification`
identifier = "046199_058819_B31F_050224_060B99_D80F_final_refined"
classification = ee.Image("...")

# GEE assets.
areaOfInterest = ee.FeatureCollection("...")

emsrWaterSurfaces = [
  ee.FeatureCollection("...")
]

destinationFolder = "..."

# **5) Configuration**

In [14]:
# `Digital Elevation Models`
demConfigs = {
  "CGIAR": {    # `SRTM Digital Elevation Data Version 4`
    "name": "CGIAR/SRTM90_V4"
  },
  "USGS": {     # `NASA SRTM Digital Elevation`
    "name": "USGS/SRTMGL1_003"
  },
  "NASA": {     # `NASA NASADEM Digital Elevation`
    "name": "NASA/NASADEM_HGT/001"
  },
  "ASTER": {    # `AG100: ASTER Global Emissivity Dataset 100-meter V003`
    "name": "NASA/ASTER_GED/AG100_003"
  }
}

demVisualization = {
  "min": 0,
  "max": 1500,
  "palette": ["black", "white"]
}

# `Dynamic World LULC v1`
dwConfig = {
  "name": "GOOGLE/DYNAMICWORLD/V1"
}

dwVisualization = {
  "min": 0,
  "max": 8,
  "bands": ["label"],
  "palette": [
    "419BDF", "397D49", "88B053",
    "7A87C6", "E49635", "DFC35A",
    "C4281B", "A59B8F", "B39FE1"
  ]
}

# `Classification`
classVisualization = {
  "min": 0,
  "max": 2,
  "palette": ["deb887", "C60404", "45b6fe"]
}

# GEE assets
demConfig = demConfigs[demProvider]

# **6) Data Processing**

In [15]:
# Define the projection of interest.

projection = ee.Projection("EPSG:4326").atScale(10)

# Load, filter and process the rasters.

# `Digital Elevation`
elevation = ee.Image(demConfig["name"]).clipToCollection(areaOfInterest)
steepAreas = ee.Terrain.slope(elevation).gte(steepnessThreshold)

# `Dynamic World Land Cover`
dwMosaic = ee.ImageCollection(dwConfig["name"])  \
  .filterDate(dwStartDate, dwEndDate)         \
  .filterBounds(areaOfInterest)               \
  .select(["label"])                          \
  .mode()                                     \
  .clipToCollection(areaOfInterest)           \
  .reproject(projection)                      \

# Create water mask.
dwWaterMask = dwMosaic.eq(0)

# Process surfaces catalog.

# Flatten water surfaces.
emsrWaterSurfaces = ee.FeatureCollection(emsrWaterSurfaces) \
  .flatten()                                                \
  .map(lambda vector: vector.set("label", 1))

# Create water mask.
emsrWaterMask = emsrWaterSurfaces     \
  .reduceToImage(**{
    "properties": ["label"],
    "reducer": ee.Reducer.first()
  })                                  \
  .unmask()                           \
  .rename("label")                    \
  .clipToCollection(areaOfInterest)   \
  .reproject(projection)

emsrNonWaterMask = emsrWaterMask.Not()

# Identify misclassifed surfaces.
misclassifiedFloodSurfaces = classification.eq(1).bitwiseAnd(emsrWaterMask)
misclassifiedWaterSurfaces = classification.eq(2).bitwiseAnd(emsrNonWaterMask)

# Refine the classification by correcting misclassified areas.
steepAreas = steepAreas.where(emsrWaterMask, 0)

refined = classification                  \
  .where(steepAreas, 0)                   \
  .where(misclassifiedWaterSurfaces, 0)   \
  .where(misclassifiedFloodSurfaces, 2)

# Refine classification with a Majority filter.

# Count patch sizes.
patchSize = refined.connectedPixelCount(50, False)

# Construct the filter.
filtered = refined.focal_mode(**{
  "radius": 30,
  "kernelType": "square",
  "units": "meters",
})

# Replace small patches with filtered values.
refined = refined.where(patchSize.lt(30), filtered)

# **8) Map Visualization**

In [17]:
Map = geemap.Map()
Map.centerObject(areaOfInterest)

Map.addLayer(dwMosaic, dwVisualization, "rasters: DW LULC")

Map.addLayer(elevation, demVisualization, "rasters: elevation")
Map.addLayer(steepAreas, {min:0, max:1}, "rasters: steep areas")

Map.addLayer(classification, classVisualization, "rasters: classification")
Map.addLayer(refined, classVisualization, "rasters: classification (refined)")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# **9) Data Export**

In [ ]:
exportTask = ee.batch.Export.image.toAsset(**{
  "image": refined,
  "assetId": [destinationFolder, identifier].join("/"),
  "description": identifier,
  "maxPixels": 1e13,
  "crs": projectionCRS,
  "scale": projectionScale,
  "region": areaOfInterest.geometry(),
})

# Submit the tasks.
exportTask.start()

In [ ]:
# Monitor the classifier tasks.
export_tasks_viewer([exportTask.id])

Task_Id                   Task_State    Task_Type       Task_Attempt  Task_Description                   Queue_Time    Execution_Time    Completion_Time  Error_Message
67AODJDMDOY5AXQBTWHSMPD7  COMPLETED     EXPORT_IMAGE               1  farkadona_ground_truth_to_asset        14.477           143.944            158.421


-End of Notebook-